In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
train = pd.read_csv('train.csv')
holidays = pd.read_csv('holidays_events.csv')

In [12]:
#this is Lars's code to add in a time variable
dates_dt = pd.to_datetime(train['date'])
dates_dt_min = dates_dt.min()
dates_days = (dates_dt - dates_dt_min).dt.days

display(dates_days)
train['time'] = dates_days
display(train)

#we also add a time variable to the holidays dataframe
holidays_days = (pd.to_datetime(holidays['date']) - dates_dt_min).dt.days

display(holidays_days)
holidays['time'] = holidays_days
display(holidays)

0             0
1             0
2             0
3             0
4             0
           ... 
3000883    1687
3000884    1687
3000885    1687
3000886    1687
3000887    1687
Name: date, Length: 3000888, dtype: int64

,id,date,store_nbr,family,sales,onpromotion,time
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,0
1,1,2013-01-01,1,BABY CARE,0.000,0,0
2,2,2013-01-01,1,BEAUTY,0.000,0,0
3,3,2013-01-01,1,BEVERAGES,0.000,0,0
4,4,2013-01-01,1,BOOKS,0.000,0,0
...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,1687
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,1687
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,1687
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,1687


0      -305
1      -275
2      -264
3      -262
4      -255
       ... 
345    1816
346    1817
347    1818
348    1819
349    1820
Name: date, Length: 350, dtype: int64

,date,type,locale,locale_name,description,transferred,time
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,-305
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,-275
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,-264
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,-262
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,-255
...,...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False,1816
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False,1817
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False,1818
348,2017-12-25,Holiday,National,Ecuador,Navidad,False,1819


First we define a function to check if a day is a holiday or not for a specific store, dependent on the region and locale of the store.

In [13]:
#write a function to check if a day is a holiday or event for a specific store
def is_holiday(t, st):
    #check if the day shows up in the holidays_events dataset
    if np.isin(t,holidays['time']):
        #if it is record the index of the holiday
        holidayindex = np.where(holidays['time'] == t)[0][0]
        storeindex = np.where(stores['store_nbr'] == st)[0][0]
        #check if the holiday is transferred away from this day
        if holidays['transferred'][holidayindex]:
            #if the holiday is transferred treat it like a normal day
            return 'No'
        else:
            #check if the holiday is transferred from another holiday
            if holidays['type'][holidayindex] == 'Transfer':
                #if it is transferred treat it like a holiday
                d = 'Holiday'
            else:
                d = holidays['type'][holidayindex]
            #check if the holiday affects the region
            if holidays['locale'][holidayindex] == 'National':
                return d
            else:
                #check if the holiday is regional or local
                if holidays['locale'][holidayindex] == "Regional":
                    #check if the store is in the region
                    if stores['state'][storeindex] == holidays['locale_name'][holidayindex]:
                        return d
                    else:
                        return 'No'
                else:
                    #check if the store is in the locality of the local holiday
                        if stores['city'][storeindex] == holidays['locale_name'][holidayindex]:
                            return d
                        else:
                            return 'No'
    #if it is not a holiday return 'No'
    else:
        return 'No'

                    

We then test that it works

In [14]:
print(is_holiday(t=341, st = 38))
print(is_holiday(t=341, st = 3))
print(is_holiday(t=0, st = 3))

Holiday
No
Holiday


Now we add holidays to the test dataframe

In [15]:
vector_holiday = np.vectorize(is_holiday)

holiday_ar = vector_holiday(train['time'],train['store_nbr'])

display(holiday_ar)

array(['Holiday', 'Holiday', 'Holiday', ..., 'No', 'No', 'No'],
      dtype='<U10')

In [16]:
train['holiday'] = holiday_ar
display(train)

,id,date,store_nbr,family,sales,onpromotion,time,holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,0,Holiday
1,1,2013-01-01,1,BABY CARE,0.000,0,0,Holiday
2,2,2013-01-01,1,BEAUTY,0.000,0,0,Holiday
3,3,2013-01-01,1,BEVERAGES,0.000,0,0,Holiday
4,4,2013-01-01,1,BOOKS,0.000,0,0,Holiday
...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,1687,No
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,1687,No
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,1687,No
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,1687,No
